In [ ]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using Piccolo
using LinearAlgebra

# Plots
using CairoMakie

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 1
const n_levels = 2

t_f = 50 * ns
n_steps = 101
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

In [ ]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
Rx(theta)  = kron(Paulis["I"],exp(-im * theta * Paulis["X"]/2))
Ry(theta)  = kron(Paulis["I"],exp(-im * theta * Paulis["Y"]/2))
Rz(theta)  =  kron(Paulis["I"],exp(-im * theta * Paulis["Z"]/2));

In [ ]:
a = [0 1; 0 0]
ad = transpose(a)
a0  = kron(a,Paulis["I"])
ad0  = kron(ad,Paulis["I"])
a1  = kron(Paulis["I"],a)
ad1  = kron(Paulis["I"],ad);

In [ ]:
H_drift = [ ]
H_drives = [
    ad0 * a0,
    ad1 * a1,
    a0+ad0,
    im * (a0-ad0),
    a1+ad1,
    im * (a1-ad1),
    ad0 * a1 + a0 * ad1
]
system = QuantumSystem(H_drives);

In [ ]:
target=Rx(pi/2)

In [ ]:
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 100.
R = .001

# Add control bounds
a_bound = 2 * π * 500 * MHz
dda_bound = .02

problem = UnitarySmoothPulseProblem(
    system,
    target,
    n_steps,
    Δt;
    a_bound=a_bound,
    dda_bound=dda_bound,
    Q=Q,
    R=R,
    verbose=true,
    hessian_approximation=true,
    pade_order=4,
    free_time=true,
    timesteps_all_equal=false,
    subspace=[1,2,3,4],
    max_iter=PICO_max_iter,
)

In [ ]:
infidelities

In [ ]:
solve!(problem)

In [ ]:
result = copy(problem.trajectory)
states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2));

In [ ]:
unitary_infidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target))

In [ ]:
rollout_states = unitary_rollout(result, system; integrator=exp);

# .- the dot applies the operation elementwise
ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
maximum(ΔUs)     

In [ ]:
unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))

In [ ]:
ts = accumulate(+, timesteps(result)) .- timesteps(result)[1]
as = result[:Ũ⃗]

f = Figure()
ax = Axis(f[1, 1], xlabel="ns", ylabel="GHz")

for i in 1:size(as, 1)
    lines!(ax, ts, as[i, :], label="$i")
end
f